# Exercice

## Calculate the mean altitude per department

For some reason it is quite difficult to find the mean altitude of each department in France on the web. So let's compute it and check out which department is the highest on average.

For those who don't know, France is divided into several administrative levels, the main ones being:   

- Régions (13 + 5 overseas)
- Départements (96 + 5 overseas)
- Communes (about 35,000), which can be translated as "municipalities", "towns" or "cities".

## Instructions

- Go on the web and find a dataset that allows you to do this.
- Then use pandas to load the dataset and perform the necessary calculations.
- Verify that your results are the same than the file `mean_altitude_dept_output.csv` which is located in the same folder as this notebook. Results may vary slightly depending on the dataset you used, but the differences should be minimal.

In [ ]:
import pandas as pd

# Code here!


In [8]:
import pandas as pd

d = {
    "code_insee": ["string", "insee_code"],
    "nom_standard": ["string", "name"],
    "dep_code": ["string", "dep_code"],
    "dep_nom": ["string", "dep_name"],
    "superficie_km2": ["float32", "area_km2"],
    "altitude_moyenne": ["float32", "mean_altitude_m"],
}

df = pd.read_csv(
    "data/communes-france-2025.csv.gz",
    compression="gzip",
    usecols=d.keys(),
    dtype={k: v[0] for k, v in d.items()}
).rename(columns={k: v[1] for k, v in d.items()})

df

,insee_code,name,dep_code,dep_name,area_km2,mean_altitude_m
0,01001,L'Abergement-Clémenciat,01,Ain,16.0,242.0
1,01002,L'Abergement-de-Varey,01,Ain,9.0,483.0
2,01004,Ambérieu-en-Bugey,01,Ain,24.0,379.0
3,01005,Ambérieux-en-Dombes,01,Ain,16.0,290.0
4,01006,Ambléon,01,Ain,6.0,589.0
...,...,...,...,...,...,...
34930,97613,M'Tsangamouji,976,Mayotte,22.0,96.0
34931,97614,Ouangani,976,Mayotte,18.0,175.0
34932,97615,Pamandzi,976,Mayotte,4.0,52.0
34933,97616,Sada,976,Mayotte,11.0,130.0


In [7]:
import pandas as pd

dtypes = {
    "code_insee": "string",
    "nom_standard": "string",
    "dep_code": "string",
    "dep_nom": "string",
    "superficie_km2": "float32",
    "altitude_moyenne": "float32",
}

df = pd.read_csv(
    "data/communes-france-2025.csv.gz",
    compression="gzip",
    usecols=dtypes.keys(),
    dtype=dtypes
).rename(columns={
    "code_insee": "insee_code",
    "nom_standard": "standard_name",
    "dep_code": "dep_code",
    "dep_nom": "dep_name",
    "superficie_km2": "area_km2",
    "altitude_moyenne": "mean_altitude_m"})

df

,insee_code,standard_name,dep_code,dep_name,area_km2,mean_altitude_m
0,01001,L'Abergement-Clémenciat,01,Ain,16.0,242.0
1,01002,L'Abergement-de-Varey,01,Ain,9.0,483.0
2,01004,Ambérieu-en-Bugey,01,Ain,24.0,379.0
3,01005,Ambérieux-en-Dombes,01,Ain,16.0,290.0
4,01006,Ambléon,01,Ain,6.0,589.0
...,...,...,...,...,...,...
34930,97613,M'Tsangamouji,976,Mayotte,22.0,96.0
34931,97614,Ouangani,976,Mayotte,18.0,175.0
34932,97615,Pamandzi,976,Mayotte,4.0,52.0
34933,97616,Sada,976,Mayotte,11.0,130.0


In [17]:
total_area_per_department = df.groupby(["dep_code", "dep_name"]).agg(
    department_area_km2=("area_km2", "sum")
).sort_values("department_area_km2", ascending=False).reset_index()

total_area_per_department

,dep_code,dep_name,department_area_km2
0,973,Guyane,83824.0
1,33,Gironde,10061.0
2,40,Landes,9261.0
3,21,Côte-d'Or,8734.0
4,71,Saône-et-Loire,8513.0
...,...,...,...
96,976,Mayotte,366.0
97,94,Val-de-Marne,242.0
98,93,Seine-Saint-Denis,235.0
99,92,Hauts-de-Seine,178.0


In [18]:
df = df.merge(
    total_area_per_department,
    on=["dep_code", "dep_name"],
    how="left")

In [19]:
df['area_share'] = df['area_km2'] / df['department_area_km2']
df['mean_altitude_weighted'] = df['mean_altitude_m'] * df['area_share']

In [20]:

df.groupby(["dep_code", "dep_name"], as_index=False).agg(
    mean_altitude_m=("mean_altitude_weighted", lambda x: int(x.sum()))
).sort_values("mean_altitude_m", ascending=False).reset_index(drop=True)


,dep_code,dep_name,mean_altitude_m
0,05,Hautes-Alpes,1661
1,73,Savoie,1570
2,04,Alpes-de-Haute-Provence,1134
3,74,Haute-Savoie,1130
4,06,Alpes-Maritimes,1110
...,...,...,...
96,85,Vendée,55
97,33,Gironde,43
98,17,Charente-Maritime,37
99,75,Paris,35
